In [1]:
import duckdb

In [2]:
# Connect to (or create) a DuckDB file
con = duckdb.connect("crashes.duckdb")

# Create a table inside DuckDB from the Parquet file
con.execute("""
    CREATE OR REPLACE TABLE crashes AS 
    SELECT 
        BOROUGH,
        EXTRACT(YEAR FROM "CRASH DATE") AS YEAR,
        COALESCE("NUMBER OF PERSONS KILLED", 0) AS PERSONS_KILLED,
        COALESCE("NUMBER OF PERSONS INJURED", 0) AS PERSONS_INJURED
    FROM 'crashes.parquet'
""")

In [3]:
print(con.execute("SELECT COUNT(*) FROM crashes").fetchall())

[(1989065,)]


In [ ]:
# create a persistent connection to the DuckDB file
con = duckdb.connect("crashes.duckdb")

In [ ]:
df = con.execute("SELECT * FROM crashes").df()
df.head()

In [ ]:
df.shape

In [ ]:
total_crashes = con.execute("SELECT COUNT(*) FROM crashes").fetchall()[0]
total_crashes

In [ ]:
con.execute("""
    SELECT 
        COUNT(*) AS total_collisions, 
        SUM(PERSONS_KILLED) AS persons_killed, 
        SUM(PERSONS_INJURED) AS persons_injured
    FROM crashes
""").fetchone()

In [ ]:
kpi_query = con.execute("""
    SELECT 
        COUNT(*) AS total_collisions, 
        SUM(PERSONS_KILLED) AS persons_killed, 
        SUM(PERSONS_INJURED) AS persons_injured
    FROM crashes
""").fetchone()

In [ ]:
total_collisions, persons_killed, persons_injured = map(int, kpi_query)

In [ ]:
persons_injured